Notes:
- The default limit is 10/s for all users.
- Query-based APIs CANNOT increase in frequency and are limited to 10 requests per second (WebSocket is recommended for data reception).
- With V5, users can request the last 500 orders in real-time.
- Users can cancel all Derivatives orders settled by the same currency with settleCoin.
- API V5 support the query of funding rate changes, which allows users to conduct historical funding backtests.
- Previous APIs returned data as E2 and E8. API V3 and V5 return data as actual values, eliminating the need for users to implement conversion logic.

In [13]:
import datetime
import requests
import numpy as np
import pandas as pd

In [60]:
API_KEY = 'P7cr5wnUHvI9El1l4G'
API_SECRET = 'C61cVcaooMXHcHex8WjcGWgtuYWx0D91QR7E'

# Bybit server time

In [38]:
url = 'https://api.bybit.com/v5/market/time'
response = requests.get(url)
cur_time = response.json()['time']
print('Bybit server time:', datetime.datetime.fromtimestamp(cur_time / 1000))

Bybit server time: 2023-08-18 16:20:37.657000


# Market endpoint

In [43]:
url = 'https://api.bybit.com/v5/market/kline'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'interval': 60, 'start': cur_time - 3600000}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'symbol': 'BTCUSDT',
  'category': 'linear',
  'list': [['1692367200000',
    '26222.7',
    '26267.3',
    '26100.5',
    '26222.3',
    '7264.495',
    '190199538.3872']]},
 'retExtInfo': {},
 'time': 1692369414129}

- list[0]: startTime (string):	Start time of the candle (ms)
- list[1]: openPrice (string): Open price
- list[2]: highPrice (string): Highest price
- list[3]: lowPrice	(string): Lowest price
- list[4]: closePrice (string): Close price.
- list[5]: volume (string): Trade volume. Unit of contract: pieces of contract. Unit of spot: quantity of coins
- list[6]: turnover (string): Turnover. Unit of figure: quantity of quota coin

In [44]:
url = 'https://api.bybit.com/v5/market/instruments-info'
params = {'category': 'linear', 'symbol': 'BTCUSDT'}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'contractType': 'LinearPerpetual',
    'status': 'Trading',
    'baseCoin': 'BTC',
    'quoteCoin': 'USDT',
    'launchTime': '1584230400000',
    'deliveryTime': '0',
    'deliveryFeeRate': '',
    'priceScale': '2',
    'leverageFilter': {'minLeverage': '1',
     'maxLeverage': '100.00',
     'leverageStep': '0.01'},
    'priceFilter': {'minPrice': '0.10',
     'maxPrice': '199999.80',
     'tickSize': '0.10'},
    'lotSizeFilter': {'maxOrderQty': '100.000',
     'minOrderQty': '0.001',
     'qtyStep': '0.001',
     'postOnlyMaxOrderQty': '1000.000'},
    'unifiedMarginTrade': True,
    'fundingInterval': 480,
    'settleCoin': 'USDT',
    'copyTrading': 'normalOnly'}],
  'nextPageCursor': ''},
 'retExtInfo': {},
 'time': 1692369420668}

In [46]:
url = 'https://api.bybit.com/v5/market/orderbook'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'limit': 5}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'s': 'BTCUSDT',
  'b': [['26250', '1.082'],
   ['26249.9', '0.011'],
   ['26249.8', '0.001'],
   ['26249.7', '0.001'],
   ['26249.6', '0.001']],
  'a': [['26250.1', '70.278'],
   ['26250.2', '0.01'],
   ['26250.9', '0.549'],
   ['26251', '2.299'],
   ['26251.1', '21.298']],
  'ts': 1692369657846,
  'u': 104547421},
 'retExtInfo': {},
 'time': 1692369657905}

- limit:
    - spot: [1, 50]. Default: 1.
    - linear&inverse: [1, 200]. Default: 25.
    - option: [1, 25]. Default: 1.
- ts (integer): The timestamp (ms) that the system generates the data.
- u (integer): Update ID, is always in sequence.

In [52]:
# Query for the latest price snapshot, best bid/ask price, and trading volume in the last 24 hours.
url = 'https://api.bybit.com/v5/market/tickers'
params = {'category': 'linear', 'symbol': 'BTCUSDT'}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'lastPrice': '26226.30',
    'indexPrice': '26238.96',
    'markPrice': '26226.71',
    'prevPrice24h': '28355.60',
    'price24hPcnt': '-0.075092',
    'highPrice24h': '28511.20',
    'lowPrice24h': '24200.90',
    'prevPrice1h': '26339.40',
    'openInterest': '58606.188',
    'openInterestValue': '1537047496.88',
    'turnover24h': '9518868638.9617',
    'volume24h': '354908.4760',
    'fundingRate': '0',
    'nextFundingTime': '1692374400000',
    'predictedDeliveryPrice': '',
    'basisRate': '',
    'deliveryFeeRate': '',
    'deliveryTime': '0',
    'ask1Size': '0.114',
    'bid1Price': '26226.20',
    'ask1Price': '26226.30',
    'bid1Size': '103.846',
    'basis': ''}]},
 'retExtInfo': {},
 'time': 1692370179633}

In [56]:
url = 'https://api.bybit.com/v5/market/funding/history'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'limit': 3}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'fundingRate': '-0.00000492',
    'fundingRateTimestamp': '1692345600000'},
   {'symbol': 'BTCUSDT',
    'fundingRate': '0.00006393',
    'fundingRateTimestamp': '1692316800000'},
   {'symbol': 'BTCUSDT',
    'fundingRate': '0.0001',
    'fundingRateTimestamp': '1692288000000'}]},
 'retExtInfo': {},
 'time': 1692370473472}

In [55]:
url = 'https://api.bybit.com/v5/market/recent-trade'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'limit': 2}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'execId': 'bad8311e-902a-5f24-b2d8-237b11fbd7a4',
    'symbol': 'BTCUSDT',
    'price': '26194.90',
    'size': '0.005',
    'side': 'Sell',
    'time': '1692370343377',
    'isBlockTrade': False},
   {'execId': 'd38ea554-31e4-55dc-af1a-c2bee1b70396',
    'symbol': 'BTCUSDT',
    'price': '26194.90',
    'size': '0.143',
    'side': 'Sell',
    'time': '1692370343347',
    'isBlockTrade': False}]},
 'retExtInfo': {},
 'time': 1692370343450}

limit:
- spot: [1,60], default: 60.
- others: [1,1000], default: 500

In [57]:
url = 'https://api.bybit.com/v5/market/open-interest'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'intervalTime': '15min', 'limit': 2}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'symbol': 'BTCUSDT',
  'category': 'linear',
  'list': [{'openInterest': '58615.33500000', 'timestamp': '1692369900000'},
   {'openInterest': '57958.54900000', 'timestamp': '1692369000000'}],
  'nextPageCursor': 'lastid%3D64884144%26lasttime%3D1692369000'},
 'retExtInfo': {},
 'time': 1692370559803}

- intervalTime (string): Interval. 5min,15min,30min,1h,4h,1d
- limit (integer): Limit for data size per page. [1, 200]. Default: 50

In [58]:
# - The data is hourly.
# - If both startTime and endTime are not specified, it will return the most recent 1 hours worth of data.
# - startTime and endTime are a pair of params. Either both are passed or they are not passed at all.
# - This endpoint can query the last 2 years worth of data, but make sure [endTime - startTime] <= 30 days.

url = 'https://api.bybit.com/v5/market/historical-volatility'
params = {'category': 'option', 'baseCoin': 'BTC', 'period': 30}
response = requests.get(url, params=params)
response.json()

{'retCode': 0,
 'retMsg': 'SUCCESS',
 'category': 'option',
 'result': [{'period': 30, 'value': '0.28227017', 'time': '1692370800000'}]}

# Account endpoint

In [67]:
BASE_URL = 'https://api.bybit.com'
endpoint = '/v5/account/wallet-balance'
url = BASE_URL + endpoint
params = {'accountType': 'UNIFIED'}
response = requests.get(url, params=params)
response

<Response [401]>

# Order endpoint

In [ ]:
BASE_URL = 'https://api-testnet.bybit.com'
endpoint = '/v5/order/create'
params = {'category': 'linear', 'symbol': 'BTCUSDT', 'side': 'Buy', 'orderType': 'Market', 'qty': '0.001'}